In [ ]:
import os
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

#import lightning as L
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import tqdm
import matplotlib.pyplot as plt
from collections import Counter
import pickle
import re
import copy
import pprint
import time

MAX_NO_OF_SPEAKERS = 8
MAX_DIALOGUE_LEN   = 33
original_labels    = ['abuse', 'adoration', 'annoyance', 'awkwardness', 'benefit', 'boredom', 'calmness', 'challenge', 'cheer', 'confusion', 'curiosity', 'desire', 'excitement', 'guilt', 'horror', 'humour', 'impressed', 'loss', 'nervousness', 'nostalgia', 'pain', 'relief', 'satisfaction', 'scold', 'shock', 'sympathy', 'threat']
train_count        = [31, 190, 1051, 880, 220, 78, 752, 214, 534, 486, 545, 180, 867, 216, 280, 153, 257, 351, 398, 65, 36, 173, 136, 94, 372, 209, 263]

EMOTIONS           = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
#OUTPUT_MASK        = pickle.load(open('dump_files/mask/top10_mask0_dict.pkl', 'rb'))['mask_tensor']

#@title Import Files { display-mode: "both" }
sent_model = 'roberta-base-nli-stsb-mean-tokens' #@param {type:"string"}

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from torch import nn, optim


print('tr version', transformers.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device => ",device, ' torch ', torch.__version__)

# for finetuned sentence embeddings
from transformers import RobertaModel, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup

In [ ]:
class EmotionClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EmotionClassifier, self).__init__()
        self.bert = RobertaModel.from_pretrained('roberta-base')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    def forward(self, input_ids, attention_mask):
        op = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        output = self.drop(op[1])
        return self.out(output), op[1]

# load finetuned roberta model
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_finetuned = EmotionClassifier(7).to(device)
#roberta_tf_checkpoint = torch.load('dump_files/finetuned/best_model_state_roberta.bin', map_location=torch.device(device))
#roberta_finetuned.load_state_dict(roberta_tf_checkpoint)
print('model loaded')


# Helper functions



In [ ]:
train_csv = pd.read_json("Code/EDiReF-Train-Data/Task 3/MELD_train_efr.json")
dev_csv = pd.read_json("Code/EDiReF-Train-Data/Task 3/MELD_train_efr.json")
test_csv = pd.read_json("Code/EDiReF-Train-Data/Task 3/MELD_train_efr.json")


In [ ]:
train_csv['emotions'][0:20]

In [ ]:
train_df = pd.DataFrame(train_csv)

In [ ]:
print(train_df['emotions'][0], train_df['speakers'][0])

In [ ]:
dummies = pd.get_dummies(EMOTIONS)
dummies['anger']

In [ ]:
listaEmo = []
for i in train_df['emotions']:
    listtemp = []
    for j in i:
        listtemp.append(dummies[j])
    listaEmo.append(listtemp)

In [ ]:
train_df['emotions'] = listaEmo
train_df

In [ ]:
train_df['speakers']

In [ ]:
listSpk = []
for i in train_df['speakers']:
    dic = {}
    tempskp = []
    count = 0
    for speaker in i:
        if speaker not in dic:
            count = count + 1
            dic[speaker] = count
        tempskp.append(dic[speaker])
    listSpk.append(tempskp)

In [ ]:
train_df['speakers'] = listSpk
train_df['speakers']

In [ ]:
i = 0
sentence_embeddings = []
    # sent_emb = model.encode('')
while i < len(train_df):
    utt = train_df['utterances'][i]
    encodings = roberta_tokenizer.encode_plus(utt, max_length=100, padding = 'max_length', add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True, truncation=True, return_tensors='pt').to(device)
    utt_emb = roberta_finetuned(encodings['input_ids'], encodings['attention_mask'])[1].detach().tolist()[0]
    utt_emb = np.round(utt_emb, decimals = 10)
    # utt_emb = model.encode(utt)
    sent_emb = utt_emb
    i += 1
    sentence_embeddings.append(copy.deepcopy(sent_emb))


In [ ]:
train_df['sentence_embeddings'] = sentence_embeddings
df_sent = pd.DataFrame(sentence_embeddings)

In [ ]:
csvread = pd.read_csv('/kaggle/input/djejeje/out.csv',names=["Valence", "Arousal", "Dominance"])
print(csvread)

In [ ]:
import re
from collections import defaultdict

track = defaultdict(list)

for i in train_df['utterances']:
    for sentence in i:
        sentence = sentence.lower().split()
        for word in sentence:
            cleaned_word = re.sub(r'[^a-zA-Z]', '', word)
            if cleaned_word in csvread.index and cleaned_word not in track:
                track[cleaned_word].append(csvread['Valence'][cleaned_word])
                track[cleaned_word].append(csvread['Arousal'][cleaned_word])
                track[cleaned_word].append(csvread['Dominance'][cleaned_word])
                

# Ahora, track contendrá las palabras limpias como claves y listas de diccionarios como valores, 
# donde cada diccionario contiene las propiedades Valence, Arousal y Dominance para esa palabra.


In [ ]:
track

In [ ]:
valen = []
aros = []
domi = []
for i in train_df['utterances']:
    listVal = []
    listAro = []
    listDom = []
    for sentence in i:
        valence_sen = []
        arousal_sen = []
        dominance_sen = []
        sentence = sentence.lower().split()
        for word in sentence:
            cleaned_word = re.sub(r'[^a-zA-Z]', '', word)
            if cleaned_word in track:
                val, aro, dom = track[cleaned_word]
                valence_sen.append(float(val))
                arousal_sen.append(float(aro))
                dominance_sen.append(float(dom))
            else:
                valence_sen.append(0)
                arousal_sen.append(0)
                dominance_sen.append(0)
        listVal.append(valence_sen)
        listAro.append(arousal_sen)
        listDom.append(dominance_sen)
    valen.append(listVal)
    aros.append(listAro)
    domi.append(listDom)

In [ ]:
npvalen = np.array(valen)
# N-Grams, sacar unigramas, bigramas, trigramas
# Sacar promedio de las valencias
npvalen[3999]

In [ ]:
meanT = []
for i in npvalen:
    mean = []
    for j in i:
        mean.append(np.mean(j))
    meanT.append(mean)

In [ ]:
print(npvalen[3999])
train_df['utterances'][3999]

In [ ]:
train_df['valence'] = meanT

In [ ]:
nanlist = np.full(len(meanT[0]), np.nan)
nanlist[2] = 5
nanlist

In [ ]:
import copy
lista = []

for i in range(len(train_df['speakers'])):
    diccionario = {}
    nanlist = np.full(len(meanT[i]), np.nan)
    
    for j, speaker in enumerate(train_df['speakers'][i]):
        if speaker in diccionario:
            diccionario[speaker][j] = meanT[i][j]
        else:
            nanlist_copy = copy.copy(nanlist)  # Crea una copia independiente de nanlist
            nanlist_copy[j] = meanT[i][j]
            diccionario[speaker] = nanlist_copy

    lista.append(list(diccionario.values()))



In [ ]:
replaced_valance = [np.nan_to_num(x, nan=-1) for x in lista]
train_df['valence_speaker']=replaced_valance

In [ ]:
train_df['valence_speaker'][174]

In [ ]:
train_df

In [ ]:
import matplotlib.pyplot as plt

i = 62
# Elegir un hablante específico y sus valores
speaker1_data = list(lista[i].values())[0]
speaker2_data = list(lista[i].values())[1]
# Crear una secuencia de índices para el eje x
x = train_df['triggers'][i]

# Crear una gráfica de líneas
plt.scatter( [i for i,x in enumerate(speaker1_data)],speaker1_data)
plt.scatter( [i for i,x in enumerate(speaker2_data)],speaker2_data)
plt.scatter( [i for i,x in enumerate(x)],x)
plt.xlabel('Índice')
plt.ylabel('Valor')
plt.title('Gráfica de Líneas del Hablante')

# Mostrar la gráfica
plt.show()
print(speaker1_data, speaker2_data)


In [ ]:
class SemEvalDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.len = len(self.data)
        print(list(train_df.columns))
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        dict_x = {}
        dict_x['speaker'] = torch.tensor(self.data['speakers'][index], dtype=torch.int)
        dict_x['emotion'] = torch.tensor(self.data['emotions'][index], dtype=torch.int)
        dict_x['sentence_embeddings'] = torch.tensor(self.data['sentence_embeddings'][index], dtype=torch.float64)
        dict_x['valence'] = torch.tensor(self.data['valence'][index])

        dict_y = {}
        dict_y['triggers'] =  torch.tensor(self.data['triggers'][index], dtype=torch.float32)

        return dict_x, dict_y

In [ ]:
train_dataset = SemEvalDataset(train_df)

In [ ]:
train_df['valence'][9]

In [ ]:
train_dataset.__getitem__(9)

In [ ]:
class MELDCollate:
    def __init__(self, pad_value = 0):
        self.pad_value = pad_value
    def __call__(self, batch):
        speaker             = pad_sequence([item[0]['speaker'] for item in batch], batch_first = True)
        emotion             = pad_sequence([item[0]['emotion'] for item in batch], batch_first = True)
        sentence_embeddings = pad_sequence([item[0]['sentence_embeddings'] for item in batch], batch_first = True)
        valence             = pad_sequence([item[0]['valence'] for item in batch], batch_first = True)
        # print('\noriginal list : ',[item[0]['speaker'] for item in batch], '\n\npadded list : ', speaker)
        labels              = pad_sequence([item[1]['triggers'] for item in batch], batch_first = True)

        dict_x = { 'speaker': speaker, 'emotion':emotion,  'sentence_embeddings':sentence_embeddings, 'valence':valence}
        dict_y = {'labels': labels}

        return dict_x, dict_y

In [2]:
train_loader  = DataLoader(dataset = train_dataset, batch_size = 64, shuffle=True, collate_fn= MELDCollate())

NameError: name 'DataLoader' is not defined

In [1]:
desired_batch_index = 6
for i, batch in enumerate(train_loader):
    if i == desired_batch_index:
        # 'batch' contendrá el batch en el índice especificado
        print(f"Batch {i}:")
        print(batch['emotions'])
        break

NameError: name 'train_loader' is not defined